In [3]:
import pandas as pd
import numpy as np

In [4]:
#read csv files in data
train = pd.read_csv('data/train.csv')
test= pd.read_csv('data/test.csv')

In [5]:
#only show train that has dtype=object
train.select_dtypes(include=['object']).columns

Index(['product_type', 'sub_area', 'culture_objects_top_25',
       'thermal_power_plant_raion', 'incineration_raion',
       'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion',
       'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion',
       'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology'],
      dtype='object')

In [6]:
#convert data types where coluhmns have values yes and no to 1 and 0
train = train.replace({'yes': 1, 'no': 0})
test = test.replace({'yes': 1, 'no': 0})

In [7]:
#make surre all yes and no are covnerted
train.select_dtypes(include=['object']).columns

Index(['product_type', 'sub_area', 'ecology'], dtype='object')

In [8]:
#  Import the necessary libraries
from sklearn.preprocessing import LabelEncoder

In [9]:

#Create an instance of LabelEncoder
lbl = LabelEncoder()

In [10]:

#Fit the label encoder on the combined data of train and test datasets
lbl.fit(list(train['ecology'].values) + list(test['ecology'].values))

LabelEncoder()

In [11]:

#Convert the 'ecology' column in the train dataset
train['ecology'] = lbl.transform(list(train['ecology'].values))

#Convert the 'ecology' column in the test dataset
test['ecology'] = lbl.transform(list(test['ecology'].values))

In [12]:
#do label encoding for prodcut type as well
lbl.fit(list(train['product_type'].values) + list(test['product_type'].values))
train['product_type'] = lbl.transform(list(train['product_type'].values))
test['product_type'] = lbl.transform(list(test['product_type'].values))

In [13]:
train.drop(columns=['sub_area'], inplace=True)
test.drop(columns=['sub_area'], inplace=True)

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [15]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [16]:
X = train.drop(columns=['price_doc'])
y = train[['price_doc']]

In [17]:
#save row id in a dataframe
row_id = test['row ID']
#drop it from test now
test = test.drop(columns=['row ID'])

In [32]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
X_new = scaler.fit_transform(X)
test_new = scaler.fit_transform(test)

In [37]:
import statsmodels.api as sm

# Add a constant column to the feature matrix
XSM = sm.add_constant(train.drop(columns=['price_doc']))

# Fit the ordinary least squares (OLS) model
model = sm.OLS(train['price_doc'], XSM).fit()

# Get the p-values of all columns
p_values = model.pvalues

# Sort the p-values in ascending order
sorted_p_values = p_values.sort_values()

# Show only the bottom 35 columns with the lowest p-values
bottom_35_p_values = sorted_p_values[:35]

# Print the bottom 35 p-values
print(bottom_35_p_values)


floor                                    5.355015e-81
full_sq                                  1.630560e-77
leisure_count_500                        3.165562e-77
industrial_km                            6.714762e-48
leisure_count_1000                       2.357339e-38
school_education_centers_top_20_raion    2.420280e-37
cafe_count_1000_price_high               8.300680e-37
mosque_count_500                         7.882503e-34
build_count_monolith                     2.046545e-31
green_part_1000                          1.165450e-30
green_zone_km                            3.146528e-26
mosque_count_1000                        5.051537e-25
cafe_count_500_price_high                8.423399e-25
build_count_1971-1995                    2.444321e-24
sport_count_2000                         8.091672e-24
green_part_500                           3.323409e-22
incineration_raion                       4.856133e-22
raion_build_count_with_builddate_info    1.295121e-20
office_sqm_1000             

In [38]:
# Use the bottom 35 features above to train the model

# Select the bottom 35 variables from the dataset
Xp = train[list(bottom_35_p_values.index)]
# Do the same for the test dataset as well
testp = test[list(bottom_35_p_values.index)]

In [39]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
testp = scaler.fit_transform(testp)
Xp = scaler.fit_transform(Xp)

In [40]:
#Xp and testp are the new datasets with only the bottom 35 features
poly = PolynomialFeatures(2, include_bias=False, interaction_only=True)
X2 = poly.fit_transform(Xp)
X2_test=poly.transform(testp)
print(X2.shape)
# print(test.shape)
print(poly.get_feature_names_out())
reg = LinearRegression().fit(X2,y)
print(reg.score(X2, y))
print(reg.coef_)
print(reg.intercept_)

(181507, 630)
['x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x6' 'x7' 'x8' 'x9' 'x10' 'x11' 'x12' 'x13'
 'x14' 'x15' 'x16' 'x17' 'x18' 'x19' 'x20' 'x21' 'x22' 'x23' 'x24' 'x25'
 'x26' 'x27' 'x28' 'x29' 'x30' 'x31' 'x32' 'x33' 'x34' 'x0 x1' 'x0 x2'
 'x0 x3' 'x0 x4' 'x0 x5' 'x0 x6' 'x0 x7' 'x0 x8' 'x0 x9' 'x0 x10' 'x0 x11'
 'x0 x12' 'x0 x13' 'x0 x14' 'x0 x15' 'x0 x16' 'x0 x17' 'x0 x18' 'x0 x19'
 'x0 x20' 'x0 x21' 'x0 x22' 'x0 x23' 'x0 x24' 'x0 x25' 'x0 x26' 'x0 x27'
 'x0 x28' 'x0 x29' 'x0 x30' 'x0 x31' 'x0 x32' 'x0 x33' 'x0 x34' 'x1 x2'
 'x1 x3' 'x1 x4' 'x1 x5' 'x1 x6' 'x1 x7' 'x1 x8' 'x1 x9' 'x1 x10' 'x1 x11'
 'x1 x12' 'x1 x13' 'x1 x14' 'x1 x15' 'x1 x16' 'x1 x17' 'x1 x18' 'x1 x19'
 'x1 x20' 'x1 x21' 'x1 x22' 'x1 x23' 'x1 x24' 'x1 x25' 'x1 x26' 'x1 x27'
 'x1 x28' 'x1 x29' 'x1 x30' 'x1 x31' 'x1 x32' 'x1 x33' 'x1 x34' 'x2 x3'
 'x2 x4' 'x2 x5' 'x2 x6' 'x2 x7' 'x2 x8' 'x2 x9' 'x2 x10' 'x2 x11'
 'x2 x12' 'x2 x13' 'x2 x14' 'x2 x15' 'x2 x16' 'x2 x17' 'x2 x18' 'x2 x19'
 'x2 x20' 'x2 x21' 'x2 x22' 'x2 x23' 'x2 

In [33]:
from sklearn.feature_selection import SelectKBest, f_regression

#Select top 30 features
selector = SelectKBest(f_regression, k=32)
X_new = selector.fit_transform(X_new, y)
X_new = pd.DataFrame(X_new)
Xtest = selector.transform(test_new)

C:\Users\hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
poly = PolynomialFeatures(2, include_bias=False, interaction_only=True)
X2 = poly.fit_transform(X_new)
X2_test=poly.transform(Xtest)
print(X2.shape)
print(test.shape)
print(poly.get_feature_names_out())
reg = LinearRegression().fit(X2,y)
print(reg.score(X2, y))
print(reg.coef_)
print(reg.intercept_)

In [41]:
price_doc = reg.predict(X2_test)

In [42]:
#Create a DataFrame for the predictions
predictions_df = pd.DataFrame(price_doc, columns=['price_doc'])

#Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'row ID', row_id)

#Save the predictions to a CSV file
predictions_df.to_csv('MLPR with interaction pvalue 35.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)